## 方式1：

* 网络模型
* 数据
* 损失函数

**调用.cuda()**

In [3]:
import torchvision
import torch
train_data = torchvision.datasets.CIFAR10(
    root = "../../../datas",
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
test_data = torchvision.datasets.CIFAR10(
    root = "../../../datas",
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
train_data_size = len(train_data)

test_data_size = len(test_data)

train_dataloader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
)

teat_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
) 

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(32,32,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32,64,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(64*4*4,64),
            torch.nn.Linear(64,10)
        )
        
    def forward(self,x):
        x = self.model(x)
        return x

#--------------------------- 修改网络模型-----------------------------
model = Model()
if torch.cuda.is_available():
    model = model.cuda()

input = torch.ones((64,3,32,32))
output = model(input)
print(input.shape)
print(output.shape) 

#--------------------------- 修改损失函数-----------------------------
loss_fn = torch.nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

total_train_step = 0

total_test_step = 0 

epoch = 10

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./project_logs")

for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    model.train()
    for data in train_dataloader:
        #--------------------------- 修改网络数据-----------------------------
        imgs,targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step+=1
        if total_train_step % 100 == 0:
            writer.add_scalar("train_loss",loss.item(),total_train_step)
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))
        
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in teat_dataloader:
            #--------------------------- 修改网络数据-----------------------------
            imgs,targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
                
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    writer.add_scalar("test_total_loss",total_test_loss,i+1)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,i+1)
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))
    
    torch.save(model,"./models/model_{}.path".format(i+1))
    print("模型{}已保存".format(i+1))
    
writer.close()

Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64, 10])
-------------第1轮训练开始------------


KeyboardInterrupt: 

## 方法二：

**device=torch.device("cpu")**

**device=torch.device("cuda")**

**device=torch.device("cuda:0")**

**device=torch.device("cuda:1")**

**.to(device)**

In [4]:
import torchvision
import torch

# --------------定义训练的设备-----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = torchvision.datasets.CIFAR10(
    root = "../../../datas",
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
test_data = torchvision.datasets.CIFAR10(
    root = "../../../datas",
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
train_data_size = len(train_data)

test_data_size = len(test_data)

train_dataloader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
)

teat_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
) 

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(32,32,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32,64,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(64*4*4,64),
            torch.nn.Linear(64,10)
        )
        
    def forward(self,x):
        x = self.model(x)
        return x

#--------------------------- 修改网络模型-----------------------------
model = Model()
model.to(device)

input = torch.ones((64,3,32,32))
output = model(input)
print(input.shape)
print(output.shape) 

#--------------------------- 修改损失函数-----------------------------
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

total_train_step = 0

total_test_step = 0 

epoch = 10

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./project_logs")

for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    model.train()
    for data in train_dataloader:
        #--------------------------- 修改网络数据-----------------------------
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step+=1
        if total_train_step % 100 == 0:
            writer.add_scalar("train_loss",loss.item(),total_train_step)
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))
        
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in teat_dataloader:
            #--------------------------- 修改网络数据-----------------------------
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            
            
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    writer.add_scalar("test_total_loss",total_test_loss,i+1)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,i+1)
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))
    
    torch.save(model,"./models/model_{}.path".format(i+1))
    print("模型{}已保存".format(i+1))
    
writer.close()

Files already downloaded and verified
Files already downloaded and verified


AssertionError: Torch not compiled with CUDA enabled